## Server Level Roles

Why would we create a custom server level role?

* allow applications to create/drop their own databases (but **only** their own databases). Unfortunately db_creator grants permissions ALTER ANY DATABASE and CREATE ANY DATABASE which would allow an application to drop _any_ database on the instance.
* allow developers access to diagnostic information without exposing other permissions.



In [ ]:
CREATE LOGIN testdbCreator WITH PASSWORD = 'fdastaert43254fgasdG';-- 'soSecure';
ALTER SERVER ROLE dbcreator ADD MEMBER testdbCreator;
GO

CREATE DATABASE neverDrop;

EXECUTE AS LOGIN = 'testdbCreator';
    CREATE DATABASE safeToDrop;

    DROP DATABASE safeToDrop;

    DROP DATABASE neverDrop;
REVERT;

CREATE DATABASE neverDrop;

EXECUTE AS LOGIN = 'croleTest';
    CREATE DATABASE safeToDrop;

    DROP DATABASE safeToDrop;

    /* we would hope this would fail: */
    DROP DATABASE neverDrop;
REVERT;

In [ ]:
CREATE SERVER ROLE [crole_CreateDB];
GO

GRANT CREATE ANY DATABASE TO [crole_CReateDB];
GO

In [ ]:
ALTER SERVER ROLE crole_CreateDB ADD MEMBER testdbCreator;
GO

CREATE DATABASE neverDrop;

EXECUTE AS LOGIN = 'testdbCreator';
    CREATE DATABASE safeToDrop;

    DROP DATABASE safeToDrop;

    /* we would hope this would fail: */
    DROP DATABASE neverDrop;
REVERT;
GO

In [ ]:
CREATE LOGIN Broseph WITH PASSWORD = 'soSecure2';
ALTER SERVER ROLE [serveradmin] ADD MEMBER Broseph;


EXECUTE AS LOGIN = 'Broseph';
BEGIN TRY
    SELECT TOP (10) wait_type, wait_time_ms 
    FROM sys.dm_os_wait_stats ORDER BY wait_time_ms DESC

    EXEC sp_configure 'show advanced options', 1;
    RECONFIGURE;
    
    EXEC sp_configure 'max degree of parallelism', 24;
    RECONFIGURE WITH OVERRIDE;
END TRY
BEGIN CATCH
    PRINT 'You don''t have permission for that!'
END CATCH

REVERT;



In [ ]:
ALTER SERVER ROLE [serveradmin] DROP MEMBER Broseph;

EXEC sp_configure 'max degree of parallelism', 4;
EXEC sp_configure 'show advanced options', 0;
RECONFIGURE;
GO

CREATE SERVER ROLE crole_Troubleshooting;
GO

GRANT VIEW SERVER STATE TO crole_Troubleshooting;
GO

In [ ]:
ALTER SERVER ROLE crole_Troubleshooting ADD MEMBER Broseph;

EXECUTE AS LOGIN = 'Broseph';
BEGIN TRY
    SELECT TOP (10) wait_type, wait_time_ms 
    FROM sys.dm_os_wait_stats ORDER BY wait_time_ms DESC

    EXEC sp_configure 'show advanced options', 1
    RECONFIGURE
    
    EXEC sp_configure 'max degree of parallelism', 24
    RECONFIGURE WITH OVERRIDE
END TRY
BEGIN CATCH
    PRINT 'You don''t have permission for that!'
END CATCH

REVERT;

## Database Level Roles

Custom database roles allow us to provide very granular permissions to the data. We can limit access or prevent users from updating certain tables.
